In [99]:
import os
import numpy as np
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
import patsy
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from sklearn import linear_model
import multiprocessing
import random

In [2]:
%matplotlib inline

In [166]:
ntrials = 100
nbandits = 9
choices = np.random.randint(nbandits, size=(ntrials+1))
outcomes = (np.random.randint(3, size=(ntrials+1))-1)*2

In [188]:
##compute RL values for a given alpha
def perform_RL(choices,outcomes,alpha):    
    V = np.zeros((ntrials+1,nbandits)) #initialize V
    
    if len(alpha) == 2: #seperate alphas for gains and losses
        a1,a2 = alpha
        alpha_flag = True
    else:
        alpha = alpha[0]
        alpha_flag = False    
    
    for i in range(1,ntrials+1): 
        for j in range(nbandits):

            if j == choices[i]: #chose this bandit
                delta = outcomes[i] - V[i-1,bandit] #PE
                
                if alpha_flag == True and outcomes[i] >= 0: #gain
                    V[i,j] = V[i-1,j] + a1 * delta #update
                elif alpha_flag == True and outcomes[i] < 0: #loss
                    V[i,j] = V[i-1,j] + a2 * delta #update
                else:
                    V[i,j] = V[i-1,j] + alpha* delta #update

            else: #no choice, just update value
                V[i,j] = V[i-1,j]
    return V

In [173]:
#computes subjective utility for each subjects choices
def subjective_utility(outcomes,l,rho):
    su = []
    for o in outcomes:
        if o >=0 :
            su.append(o**rho)
        else:
            su.append((-1*l) * (-1*o)**rho)
    return su

In [132]:
def softmax(values,choice_index,beta):
    values = np.exp(values*beta)
    p_choice = values[choice_index]/np.sum(values)
    return p_choice

In [190]:
#compute loss for RL model
def choice_likelihood(params,choices,outcomes,seperate_alphas):
    
    if seperate_alphas:
        alpha1 = scipy.stats.logistic.cdf(params[0]) #learning rate
        alpha2 = scipy.stats.logistic.cdf(params[1]) #learning rate
        beta = np.log(params[2])    
        V = perform_RL(choices,outcomes,(alpha1,alpha2)) #get values from RL         
    else:        
        alpha = scipy.stats.logistic.cdf(params[0]) #learning rate
        beta = np.log(params[1])    
        V = perform_RL(choices,outcomes,(alpha,)) #get values from RL 
        
    #compute LL under softmax
    ll = 0
    for i in range(1,ntrials+1):
        p_choice = softmax(V[i-1,:],choices[i],beta)
        ll += np.log(p_choice)
        
    ll = ll*-1 #minimize neg LL
    
    return ll

In [197]:
compute_su = False #compute estimates with subjective utilities instead of objective outcomes
seperate_alphas = True #compute seperate learning rates for gains and losses

#transform outcomes if you want to compute subjective utilities
if compute_su:
    outcomes = subjective_utility(outcomes,l,rho)

#initialize parameters
if seperate_alphas:
    params = [.1,.1,.1] #initialize alpha and beta
else:
    params = [.1,.1] #initialize alpha and beta

#run minimization
res = minimize(choice_likelihood,params,args=(choices,outcomes,seperate_alphas),method='BFGS', options={'disp': True})

#examine results
if seperate_alphas:
    alpha1 = scipy.stats.logistic.cdf(res.x[0]) #learning rate
    alpha2 = scipy.stats.logistic.cdf(res.x[1]) #learning rate
    beta = np.log(res.x[2])
    print "alpha for gains is " + str(alpha1)
    print "alpha for losses is " + str(alpha2)
    print "beta is " + str(beta)   
else:
    alpha = scipy.stats.logistic.cdf(res.x[0]) #learning rate
    beta = np.log(res.x[1])
    print "alpha is " + str(alpha)
    print "beta is " + str(beta)

Optimization terminated successfully.
         Current function value: 219.722458
         Iterations: 1
         Function evaluations: 15
         Gradient evaluations: 3
alpha for gains is 1.95953849504e-14
alpha for losses is 3.1431888221e-29
beta is 6.65991330117
